In [1]:
import pandas as pd
import numpy as np
import  matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['font.sans-serif'] = ['SimHei'] # 指定默认字体
plt.rcParams['axes.unicode_minus'] = False # 解决保存图像时负号‘-’显示为方块的问题

In [2]:
app = pd.read_csv(r'..\数据\application_record.csv')
cre1 = pd.read_csv(r'..\数据\credit_record.csv')

In [26]:
cre = cre1

In [27]:
cre.drop_duplicates(subset='ID')['ID'].count()

629487

In [28]:
# 去掉孩子数>=家人
app = app[app['CNT_CHILDREN']<app['CNT_FAM_MEMBERS']]

#### 去掉没有申请信息的

In [32]:
cre = cre[cre['ID'].isin(app['ID'])]
629487 - cre.drop_duplicates(subset='ID')['ID'].count() # 去掉了204305个样本

204305

In [33]:
cre.drop_duplicates(subset='ID')['ID'].count() # 还剩425182个样本

425182

#### 去掉信息不合理的--最早记录月份为1,2,3,4,5

In [34]:
pd.DataFrame(cre.drop_duplicates('ID',keep='last')['STATUS'].value_counts())

,STATUS
0,246411
X,174195
C,4408
1,128
4,21
5,19


In [16]:
last = cre.drop_duplicates('ID',keep='last') # last-每个ID的最早开户月信息
id1 = last[last['STATUS'].isin(['X','C','0'])]['ID']
cre = cre[cre['ID'].isin(id1)]
cre.drop_duplicates(subset='ID')['ID'].count() # 还剩425014个样本

425014

In [17]:
425182 - 425014 # 去掉了168个样本

168

#### 去掉最早记录信息为60个月前的

In [25]:
pd.DataFrame(cre.drop_duplicates('ID',keep='last')['MONTHS_BALANCE'].value_counts().sort_index())

,MONTHS_BALANCE
-60,3506
-59,3598
-58,3791
-57,3963
-56,4299
...,...
-4,9124
-3,8531
-2,6849
-1,6036


In [10]:
id2 = last[last['MONTHS_BALANCE'] != -60]['ID']
cre = cre[cre['ID'].isin(id2)]
cre.drop_duplicates(subset='ID')['ID'].count() # 421508个样本

421508

In [11]:
425014 - 421508 # 去掉了3506个

3506

#### vintage表生成

In [49]:
def vintage_fun():
    pass

#### vintage图生成

In [50]:
pass

#### 去除账龄小于表现期（20个月）

In [51]:
last = cre.drop_duplicates('ID',keep='last')
id3 = last[last.MONTHS_BALANCE <= -20]['ID']
cre = cre[cre['ID'].isin(id3)]
cre.drop_duplicates(subset='ID')['ID'].count() # 还剩247498个样本

247498

In [52]:
421508 - 247498 # 去掉了174010个

174010

#### 不同累计逾期的坏用户样本数

In [53]:
bad = pd.DataFrame({'逾期情况':['>1','>1','>30','>30','>60','>60','>90','>90','>120','>120'],
                    'Y':['目标Y=1','非目标Y=0','目标Y=1','非目标Y=0','目标Y=1','非目标Y=0','目标Y=1','非目标Y=0','目标Y=1','非目标Y=0',]})

n = 0
for i in ['0','1','2','3','4']:
    # 在表现期达到逾期的用户个数
    df1 = cre[cre['STATUS']==i].drop_duplicates('ID',keep='last') # 最早变坏的月份
    df2 = cre.drop_duplicates('ID',keep='last').loc[:,['ID','MONTHS_BALANCE']]
    df3 = pd.merge(df1,df2,on='ID',how='left')
    bad.loc[n*2,'样本数'] = (df3['MONTHS_BALANCE_x'] - df3['MONTHS_BALANCE_y'] <= 20).agg(sum)
    bad.loc[n*2+1,'样本数'] = 247498 - bad.loc[n*2,'样本数']
    n = n + 1
bad.set_index('逾期情况',inplace=True)
bad['总样本数'] = 247498
bad['样本占比'] = (bad['样本数']/bad['总样本数']).apply(lambda x: format(x,'.2%'))
bad

,Y,样本数,总样本数,样本占比
逾期情况,,,,
>1,目标Y=1,216048.0,247498,87.29%
>1,非目标Y=0,31450.0,247498,12.71%
>30,目标Y=1,30566.0,247498,12.35%
>30,非目标Y=0,216932.0,247498,87.65%
>60,目标Y=1,4059.0,247498,1.64%
>60,非目标Y=0,243439.0,247498,98.36%
>90,目标Y=1,1773.0,247498,0.72%
>90,非目标Y=0,245725.0,247498,99.28%
>120,目标Y=1,1278.0,247498,0.52%


#### 生成task4分箱要用的数据

In [61]:
# 给cre添加is_ovd列
df1 = cre[cre['STATUS']=='2'].drop_duplicates('ID',keep='last')
df2 = cre.drop_duplicates('ID',keep='last').loc[:,['ID','MONTHS_BALANCE']]
df3 = pd.merge(df1,df2,on='ID',how='left')
id4 = df3[(df3['MONTHS_BALANCE_x'] - df3['MONTHS_BALANCE_y']) <= 20]['ID']
cre.loc[cre['ID'].isin(id4),'is_ovd'] = 1
cre.fillna(0,inplace=True)

In [65]:
# 合并cre和app
df3 = cre.drop_duplicates('ID',keep='last').loc[:,['ID','is_ovd']]
data_model = pd.merge(df3,app,on='ID',how='left')
data_model.head()

,ID,is_ovd,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,5008806,0.0,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0
1,5008809,0.0,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
2,5008810,0.0,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
3,5008811,0.0,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
4,5008812,0.0,F,N,Y,0,283500.0,Pensioner,Higher education,Separated,House / apartment,-22464,365243,1,0,0,0,NaN,1.0


In [68]:
data_model.to_csv('data_model.csv')